In [1]:
# 라이브러리 import
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report

from xgboost import XGBClassifier


In [2]:
# 데이터 불러오기 & X, y 준비

df = pd.read_csv("stroke.csv")
target_col = "stroke" 

# X: 입력 변수(피처), y: 타깃(stroke 여부)
X = df.drop(columns=[target_col])
y = df[target_col]


In [3]:

# train / test 데이터 나누기
#  - test_size : 테스트 데이터 비율 (예: 0.2 = 20%)
#  - stratify=y : 클래스 비율을 train/test에 비슷하게 맞추기 위해 사용
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y  # 뇌졸중/비-뇌졸중 비율 유지
)


In [4]:

# 양성 클래스 비율 & scale_pos_weight 계산
#  - 여기서 '1'이 뇌졸중 발생(stroke=1)이라고 가정
#  - pos_ratio: y_train에서 1의 비율
#  - scale_pos_weight: (음성 비율 / 양성 비율)
#    -> 불균형 데이터에서 양성 클래스의 중요도를 높이는 역할

# y_train에서 양성(1)의 비율
pos_ratio = y_train.sum() / len(y_train)

# 음성(0)의 비율은 1 - pos_ratio
neg_ratio = 1 - pos_ratio

# scale_pos_weight 계산
scale_pos = neg_ratio / pos_ratio

print(f"양성(1) 비율: {pos_ratio:.4f}")
print(f"음성(0) 비율: {neg_ratio:.4f}")
print(f"계산된 scale_pos_weight: {scale_pos:.4f}")


양성(1) 비율: 0.0487
음성(0) 비율: 0.9513
계산된 scale_pos_weight: 19.5427


In [5]:
!pip install optuna

In [6]:
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

def objective(trial):
    
    params = { 
        # 모델이 배울 때 한 번에 얼마나 변화할지 (작으면 천천히 배움)
        "learning_rate": trial.suggest_float("lr", 0.01, 0.1),

        # 트리의 최대 깊이 (너무 깊으면 외워버림)
        "max_depth": trial.suggest_int("max_depth", 3, 6),

        # 만들 트리 개수 (많을수록 더 잘 배우지만 오래 걸림)
        "n_estimators": trial.suggest_int("n_estimators", 300, 900),

        # 학습에 사용할 데이터 비율 (일부만 사용해서 과적합 방지)
        "subsample": trial.suggest_float("subsample", 0.6, 0.9),

        # 트리 만들 때 사용할 변수 비율 (특정 변수에 치우치는 것 방지)
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 0.9),

        # 너무 작은 그룹은 쪼개지 않도록 하는 설정 (특이 케이스 과적합 방지)
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5),

        # 의미 있는 경우에만 분기(나누기)를 허용 (잡음 제거)
        "gamma": trial.suggest_float("gamma", 0, 1),

        # 뇌졸중 케이스(1)를 더 중요하게 보도록 가중치 주기
        "scale_pos_weight": scale_pos,

        # 모델 평가 기준을 AUC로 사용 (불균형 데이터에 좋음)
        "eval_metric": "auc",

        # 결과가 항상 똑같이 나오도록 고정 (재현성)
        "random_state": 42,

        # CPU 코어 전부 사용해서 더 빠르게 학습
        "n_jobs": -1
    }

    
    model = XGBClassifier(**params)
    auc = cross_val_score(model, X_train, y_train,
                          cv=3,
                          scoring="roc_auc").mean()
    
    return auc

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print(study.best_params)
print(study.best_value)


/home/kdt_43/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-12-18 10:44:28,672] A new study created in memory with name: no-name-72d094cb-602f-43ca-8c5d-21b22e0bed16
[I 2025-12-18 10:49:07,398] Trial 0 finished with value: 0.8228704805601708 and parameters: {'lr': 0.025835966859591844, 'max_depth': 6, 'n_estimators': 329, 'subsample': 0.858307688862531, 'colsample_bytree': 0.8139898270423931, 'min_child_weight': 1, 'gamma': 0.5186470445187267}. Best is trial 0 with value: 0.8228704805601708.
[W 2025-12-18 10:51:54,899] Trial 1 failed with parameters: {'lr': 0.0418769040401014, 'max_depth': 5, 'n_estimators': 668, 'subsample': 0.7916089550573479, 'colsample_bytree': 0.6587625099513359, 'min_child_weight': 1, 'gamma': 0.09380512450901146} because of the following error: KeyboardInterrup

KeyboardInterrupt: 

In [1]:
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

def objective(trial):
    
    # Optuna가 탐색할 하이퍼파라미터 범위 정의
    params = {
        "learning_rate": trial.suggest_float("lr", 0.01, 0.1),
        "max_depth": trial.suggest_int("max_depth", 3, 6),
        "n_estimators": trial.suggest_int("n_estimators", 300, 900),
        "subsample": trial.suggest_float("subsample", 0.6, 0.9),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 0.9),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5),
        "gamma": trial.suggest_float("gamma", 0, 1),
        "scale_pos_weight": scale_pos,  # 불균형 데이터 가중치
        "eval_metric": "auc",
        "random_state": 42,
        "n_jobs": -1,
        
        # 🔻 여기부터 GPU 관련 옵션
        # GPU에서 학습하는 트리 메서드
        "tree_method": "gpu_hist",
        # 예측도 GPU에서 수행
        "predictor": "gpu_predictor"
    }
    
    # 위에서 정의한 params로 XGBoost 모델 생성
    model = XGBClassifier(**params)
    
    # 3-fold 교차검증으로 ROC-AUC 평균 계산
    auc = cross_val_score(
        model,
        X_train,
        y_train,
        cv=3,
        scoring="roc_auc"
    ).mean()
    
    return auc

# Optuna Study 생성 (AUC를 최대화하는 방향)
study = optuna.create_study(direction="maximize")

# 하이퍼파라미터 탐색 실행 (trial 수는 필요에 따라 조절 가능)
study.optimize(objective, n_trials=50)

# 최적 파라미터와 그때의 AUC 출력
print("Best params:", study.best_params)
print("Best AUC:", study.best_value)


/home/kdt_43/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-12-18 14:52:44,855] A new study created in memory with name: no-name-56609800-cbb4-4861-a01c-b1ada12b5fb7
[W 2025-12-18 14:52:44,858] Trial 0 failed with parameters: {'lr': 0.0834143266793559, 'max_depth': 3, 'n_estimators': 789, 'subsample': 0.7931808233693001, 'colsample_bytree': 0.6547462102368415, 'min_child_weight': 2, 'gamma': 0.187862164026118} because of the following error: NameError("name 'scale_pos' is not defined").
Traceback (most recent call last):
  File "/home/kdt_43/miniconda3/lib/python3.9/site-packages/optuna/study/_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_8828/1662140942.py", line 16, in objective
    "scale_pos_weight": scale_pos,  # 불균형 데이터 가중치
NameEr

NameError: name 'scale_pos' is not defined

In [ ]:
# baseline XGBoost 모델 정의
#  - 핵심: scale_pos_weight를 우리가 계산한 값으로 설정
#  - eval_metric="auc" : 내부 평가 지표를 AUC로 사용
#  - 나머지 파라미터는 최대한 기본값 + 약간만 설정
#    (추후 튜닝을 위해 지금은 심플하게 유지)

xgb_baseline = XGBClassifier(
    random_state=42,        # 재현성을 위해 고정
    n_estimators=300,       # 기본값(100)보다 약간 늘려줌
    learning_rate=0.1,      # 기본적인 학습률
    max_depth=4,            # 너무 깊지 않게 설정 (과적합 완화)
    scale_pos_weight=scale_pos,  # **불균형 조정의 핵심**
    eval_metric="auc",      # AUC 기준으로 학습 모니터링
    n_jobs=-1               # 가능한 모든 코어 사용
)


In [ ]:
# 모델 학습
#  - X_train, y_train으로 학습
xgb_baseline.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=-1,
              num_parallel_tree=None, random_state=42, ...)

In [ ]:
# baseline 성능 확인 (ROC-AUC 중심)
#  - predict_proba 로 1(양성)일 확률을 가져와서 AUC 계산
#  - classification_report는 대략적인 성능 느낌 확인용

# 1) 확률 예측 (양성 클래스 확률만 사용)
y_proba = xgb_baseline.predict_proba(X_test)[:, 1]

# 2) ROC-AUC 계산
auc = roc_auc_score(y_test, y_proba)
print(f"✅ Baseline XGBoost (scale_pos_weight 적용) ROC-AUC: {auc:.4f}")

# 3) 디폴트 threshold(0.5) 기준으로 분류 결과 보기
y_pred_default = (y_proba >= 0.5).astype(int)
print("\n[기본 threshold=0.5 기준 분류 결과]")
print(classification_report(y_test, y_pred_default, digits=3))

# 3) 디폴트 threshold(0.2) 기준으로 분류 결과 보기
y_pred_default = (y_proba >= 0.2).astype(int)
print("\n[기본 threshold=0.2 기준 분류 결과]")
print(classification_report(y_test, y_pred_default, digits=3))


✅ Baseline XGBoost (scale_pos_weight 적용) ROC-AUC: 0.8142

[기본 threshold=0.5 기준 분류 결과]
              precision    recall  f1-score   support

           0      0.964     0.949     0.956       972
           1      0.242     0.320     0.276        50

    accuracy                          0.918      1022
   macro avg      0.603     0.634     0.616      1022
weighted avg      0.929     0.918     0.923      1022


[기본 threshold=0.2 기준 분류 결과]
              precision    recall  f1-score   support

           0      0.975     0.829     0.896       972
           1      0.149     0.580     0.237        50

    accuracy                          0.817      1022
   macro avg      0.562     0.705     0.566      1022
weighted avg      0.934     0.817     0.864      1022



In [ ]:
from xgboost import XGBClassifier

# 최적 파라미터(Optuna 결과)
best_params = {
    "learning_rate": 0.010002154204346032,
    "max_depth": 4,
    "n_estimators": 537,
    "subsample": 0.6055566573624598,
    "colsample_bytree": 0.8979287201002785,
    "min_child_weight": 5,
    "gamma": 0.7209404101015859,
}

# scale_pos_weight는 Optuna가 고정한 값 사용
# (천천히 튜닝할 때 같이 넣는 게 좋기 때문에)
best_params["scale_pos_weight"] = scale_pos

# 기타 고정 설정
best_params["eval_metric"] = "auc"
best_params["random_state"] = 42
best_params["n_jobs"] = -1


# 최종 모델 생성
model_optuna = XGBClassifier(**best_params)

# 학습
model_optuna.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8979287201002785, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', feature_types=None, gamma=0.7209404101015859,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.010002154204346032,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=537, n_jobs=-1,
              num_parallel_tree=None, random_state=42, ...)

In [ ]:
from sklearn.metrics import precision_recall_curve
import numpy as np

precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
f1_scores = (2 * precision * recall) / (precision + recall + 1e-9)

best_thr = thresholds[np.argmax(f1_scores)]
print(f"F1 기준 최적 threshold: {best_thr:.4f}")


F1 기준 최적 threshold: 0.5288


In [ ]:
y_pred_opt = (y_proba >= 0.4).astype(int)
print(classification_report(y_test, y_pred_opt, digits=3))


              precision    recall  f1-score   support

           0      0.967     0.922     0.944       972
           1      0.200     0.380     0.262        50

    accuracy                          0.895      1022
   macro avg      0.583     0.651     0.603      1022
weighted avg      0.929     0.895     0.910      1022

